In [57]:
%matplotlib inline
import numpy as np
import math
import warnings
from skimage import io
from pylab import *
from matplotlib import pylab as plt
import matplotlib.image as mpimg
from sklearn.metrics import mean_squared_error
from matplotlib.pyplot import imshow
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pydicom
from ipywidgets import FloatSlider
warnings.simplefilter("ignore")

In [58]:
def bresenham(x1, y1, x2, y2):
    x = x1
    y = y1
    points = []
    if x1 < x2:
        xi = 1
        dx = x2 - x1
    else:
        xi = -1
        dx = x1 - x2

    if y1 < y2:
        yi = 1
        dy = y2 - y1
    else:
        yi = -1
        dy = y1 - y1

    points.append([x,y])
    
    if dx > dy:
        ai = (dy - dx) * 2
        bi = dy * 2
        d = bi - dx
        while x != x2:
            if d >= 0:
                x += xi
                y += yi
                d += ai
            else:
                x += xi
                d += bi
            points.append([x,y])
    else:
        ai = (dx - dy) * 2
        bi = dx * 2
        d = bi - dy
        while y != y2:
            if d >= 0:
                x += xi
                y += yi
                d += ai
            else:
                y += yi
                d += bi
            points.append([x,y])

    return points

In [59]:
def sum_points(points, img):
    sumOfPoints = 0
    for point in points:
        sumOfPoints += img[point[0],point[1]]
    return sumOfPoints

In [60]:
def sum_points_mean(points, img):
    sumOfPoints = sum_points(points,img)
    sumOfPoints = sumOfPoints/len(points)
    return sumOfPoints

In [61]:
def coordToCart(angle, cx, cy, radius):
    x = int(math.cos(math.radians(angle)) * (radius)) + cx
    y = int(math.sin(math.radians(angle)) * (radius)) + cy
    return x,y

In [62]:
def filterKernel(size):
    img = np.zeros(size)
    center = int(size/2)
    img[center] = 1.0
    for i in range(center+1, len(img)):
        dist = i-center
        if dist % 2 == 0:
            img[i] = 0.0
            img[center-dist] = 0.0
        else:
            img[i] = (-4/math.pow(math.pi,2))/math.pow(i,2)
            img[center-dist] = (-4/math.pow(math.pi,2))/math.pow(i,2)
            
    return img

In [63]:
def doTomography(img, emiterAngles, inc_angle, emiterRange=60.0, detectors=201, withFilter=True):
    # Przyrost katów inc_angle emitera (położenie emitera), kąt z jakim wysyłane są fale (emiter_angle)
    beta = emiterRange*2
    emiters = img.copy()
    img_size = img.shape[0]
    centre = int(img_size/2)
    radius = centre-5
    
    if (detectors % 2 != 1):
        detectors += 1
    
    if withFilter:
        splotFunc = filterKernel(detectors)
    sinogram = np.zeros((len(emiterAngles),detectors))

    #petla - kazda pozycja emitera
    for angle in emiterAngles:
        emiter_x, emiter_y = coordToCart(angle, centre, centre, radius)
        emiters[emiter_y, emiter_x] = 255
    
        sin_results = np.zeros(detectors)
        #petla - kazda pozycja detektora
        
        for a in range(detectors):
            alfa = angle + 180 - beta/2 + a*beta/(detectors-1)
            detx, dety = coordToCart(alfa, centre, centre, radius)
            
            if dety < emiter_y:
                points = bresenham(detx, dety, emiter_x, emiter_y)
            else:
                points = bresenham(emiter_x, emiter_y, detx, dety)
            
            #sinogram[int(angle/inc_angle),a] = sum_points(points, img)
            sin_results[a] = sum_points(points, img)
        
        if withFilter:
            sinogram[int(angle/inc_angle),:] = np.convolve(sin_results, splotFunc, 'same')
        else:
            sinogram[int(angle/inc_angle),:] = sin_results
        
    return emiters, sinogram

In [64]:
def createOffset(img, offset=2.0):
    square_size = img.shape[0]
    side = int((int(square_size*offset) - square_size)/2)
    image = np.zeros((int(square_size*offset),int(square_size*offset)))
    for i in range(square_size):
        for j in range(square_size):
            image[i+side,j+side]= img[i,j]
    return image

In [65]:
def delOffset(img, offset = 2.0):
    square_size = img.shape[0]
    size_without_offset = int(math.ceil(square_size/offset))

    side = int((square_size - size_without_offset)/2)
    image = np.zeros((size_without_offset,size_without_offset))
    for i in range(size_without_offset):
        for j in range(size_without_offset):
            image[i,j]= img[i+side,j+side]
    return image

In [66]:
def createSquare(square_size=500):
    square = 254*np.ones((square_size,square_size), dtype=np.float64)
    return square

In [67]:
def normalize(img):
    min_value = np.min(img)
    max_value = np.max(img)
    for i in range(len(img)):
        for j in range(len(img[i])):
            img[i,j]=(img[i,j] - min_value)/(max_value-min_value) *255   
    return img

In [68]:
def meanError(imgIn, imgOut):
    if imgIn.shape!=imgOut.shape:
        return 0
    error = np.zeros(imgIn.shape[0])
    
    for i in range(imgIn.shape[0]):
        error[i] = mean_squared_error(imgIn[i], imgOut[i])
    
    return error

In [69]:
def calculateError(img,img2):
    if len(img) != len(img2):
        return (False, 0)
    if len(img[0]) != len(img2[0]):
        return (False, 0)
    error = 0.0
    for i in range(len(img)):
        for j in range(len(img[i])):
            error = error + (img[i,j]-img2[i,j])**2
    return (True, error)

In [70]:
def blurImage(img):
    print(len(img))
    newImage = np.zeros((len(img),len(img[0])),dtype=np.float64)
    print(newImage)
    for i in range(len(img)):
        for j in range(len(img[i])):
            if(i!=0 and i!=len(img)-1 and j!=0 and j!=len(img[i])-1):
                newImage[i,j] = (img[i-1,j-1] + img[i-1,j]*2 + img[i-1,j+1] + img[i,j-1]*2 + img[i,j]*3 + img[i,j+1]*2 + img[i+1,j-1] + img[i+1,j]*2 + img[i+1,j+1])/15
            else:
                newImage[i,j] = img[i,j]
    return newImage

In [71]:
def doInvTomography(sinogram, img_size, emiterAngles, inc_angle, emiterRange=60.0, detectors=201):
    # Przyrost katów inc_angle emitera (położenie emitera), kąt z jakim wysyłane są fale (emiter_angle)
    beta = emiterRange*2
    centre = int(img_size/2)
    radius = centre-5
    #print(sinogram)
    img = np.zeros((img_size, img_size))
    
    if (detectors % 2 != 1):
        detectors += 1

    #petla - kazda pozycja emitera
    for angle in emiterAngles:
        emiter_x, emiter_y = coordToCart(angle, centre, centre, radius)
        

        #petla - kazda pozycja detektora
        for a in range(detectors):
            alfa = angle + 180 - beta/2 + a*beta/(detectors-1)
            detx, dety = coordToCart(alfa, centre, centre, radius)
            #image[detx,dety] = 155
            if dety < emiter_y:
                points = bresenham(detx, dety, emiter_x, emiter_y)
            else:
                points = bresenham(emiter_x, emiter_y, detx, dety)

            for point in points:
                img[point[0], point[1]] += int(sinogram[int(angle/inc_angle), a])
    return img

In [72]:
def cutDownValues(img, cutDownValue = 15):
    for i in range(len(img)):
        for j in range(len(img[i])):
            img[i,j] = img[i,j] - cutDownValue if img[i,j] - cutDownValue > 0 else 0
    return img

In [73]:
def main(file='square', square_size=500, offset=2, 
         inc_angle=0.5, emiterRange=60.0, detectors=201, withFilter = True, upToAngle = 360):
    if file != 'square':
        img = mpimg.imread(file)
    else:
        img = createSquare(square_size)
    
    imagesCount = 5
    
    #obrazek wyjsciowy z offsetem
    fig = figure(figsize=(80,40))
    imgWithOffset = createOffset(img, offset)
    subplot(5,1,1)
    imshow(imgWithOffset, cmap='gray')
    imgSize = imgWithOffset.shape[0]
    
    #sinogram
    emiterAngles = np.linspace(0, upToAngle, upToAngle/inc_angle, endpoint=False)
    emiters, sinogram = doTomography(imgWithOffset, emiterAngles, inc_angle, emiterRange, detectors, withFilter)
    subplot(5,1,2)
    imshow(sinogram, cmap='gray')
    
    
    #obrazem z sinogramu
    imgFromSinogram = doInvTomography(sinogram, imgSize, emiterAngles, inc_angle, emiterRange, detectors)
    subplot(5,1,3)
    imshow(imgFromSinogram, cmap='gray')
    
    #obciecie offsetu
    imgToCompare = delOffset(imgFromSinogram, offset)

        
    normalized_image = normalize(imgToCompare)
    subplot(5,1,4)
    imshow(normalized_image, cmap='gray')
    
    bluredImage = blurImage(normalized_image)
    subplot(5,1,5)
    imshow(bluredImage, cmap='gray')
    print(meanError(img, normalized_image))
    print(mean_squared_error(img, normalized_image))

In [74]:
from IPython.display import clear_output
def interaction(inc_angle, emiter_angle, detectors, offset, toAngle, withFilter):
    main(file = 'ph.jpg', inc_angle=inc_angle, emiterRange=float(emiter_angle), detectors=detectors, offset = offset, upToAngle = toAngle, withFilter = withFilter)
interact_manual(interaction, inc_angle = FloatSlider(min=0.1,max=10,step=0.1,value=0.5), toAngle = widgets.IntSlider(min=30,max=360,step=10,value=180), emiter_angle = widgets.IntSlider(min=20,max=85,step=5,value=70),detectors = widgets.IntSlider(min=20,max=200,step=5,value=70), offset = FloatSlider(min=1,max=5,step=0.5,value=1.5), withFilter = True)

interactive(children=(FloatSlider(value=0.5, description='inc_angle', max=10.0, min=0.1), IntSlider(value=70, description='emiter_angle', max=85, min=20, step=5), IntSlider(value=70, description='detectors', max=200, min=20, step=5), FloatSlider(value=1.5, description='offset', max=5.0, min=1.0, step=0.5), IntSlider(value=180, description='toAngle', max=360, min=30, step=10), Checkbox(value=True, description='withFilter'), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))

<function __main__.interaction>